# Извлечение именованных сущностей.
# Задание 1

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [1]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

--2020-01-19 14:44:15--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2939556 (2.8M) [text/plain]
Saving to: ‘pristavki.csv’

pristavki.csv       100%[===================>]   2.80M  --.-KB/s    in 0.09s   

2020-01-19 14:44:16 (29.9 MB/s) - ‘pristavki.csv’ saved [2939556/2939556]



Посмотрим на данные.

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [6]:
data.shape

(9285, 1)

Давайте попробуем сначала полные варианты: Xbox 360, Xbox one, Playstation 1,2,3,4.

In [10]:
!pip install yargy

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 7.1MB 8.8MB/s 


In [0]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [0]:

Game = fact(
    'Game',
    ['name', 'second_name']
)

In [0]:
Game = fact(
    'Game',
    ['name', 'second_name', 'number']
)

COD = rule(
    morph_pipeline(['Call Of Duty', 'COD', "кол оф дюти", 'колда', 'Колда']).interpretation(Game.name),
    morph_pipeline(['Black Ops', 'Black ops', 'black ops', 'Black Ops ', 'Black ops ', 'black ops ', '4', 'Ghosts']).interpretation(Game.second_name), 
    morph_pipeline(['1', '2', '3']).interpretation(Game.number).optional()) 

Borderlands = rule(
    morph_pipeline(['Borderlands', 'Borderland', 'Borderlend', 'бордерленд', 'borderlens']).interpretation(Game.name),
    morph_pipeline([ '2', 'DLC', '3', 'длц', 'Pre-sequel', ' pre-sequel']).interpretation(Game.second_name), 
    morph_pipeline(['Mad Moxxi', 'The Zombie Island of Dr. Ned', 'The Secret Armory of General Knoxx']).interpretation(Game.number).optional()
    )

AC  = rule(
    morph_pipeline(['assassins creed', 'Assassins creed', 'Assassins Creed', 'assassin creed', 'asassins creed', 'асасин', 'асассин', 'АСАСИН']).interpretation(Game.name),
    morph_pipeline([ '1', '2', '3', '4', 'Unity', 'unity', 'юнити', 'syndicate', 'синдикат']).interpretation(Game.second_name), 
    morph_pipeline(['Black Flag', 'black flag']).interpretation(Game.number).optional())

ME  = rule(
    morph_pipeline(['Mass Effect', 'Mass effect', 'mass effect', 'mass efect', 'масс эфект', 'мас эфект', 'MASS EFFECT', 'MAS EFECT']).interpretation(Game.name),
    morph_pipeline([ '1', '2', '3', '4', 'первый']).interpretation(Game.second_name), 
    morph_pipeline(['Andromeda', 'андромеда', 'Андромеда']).interpretation(Game.number).optional())

Witcher  = rule(
    morph_pipeline(['Witcher', 'witcher', 'Ведьмак', 'ведмак', 'видмак3', 'видмак', 'ведьмак']).interpretation(Game.name),
    morph_pipeline([ '2', '3', 'Убийца королей']).interpretation(Game.second_name), 
    morph_pipeline(['Убийца королей', 'Дикая охота', 'дикая охота']).interpretation(Game.number).optional())

FIFA  = rule(
    morph_pipeline(['FIFA', 'Fifa', 'fifa', 'фифа', 'Фифа']).interpretation(Game.name),
    morph_pipeline([ '11', '12', '13','14', '15', '16']).interpretation(Game.second_name))

BF  = rule(
    morph_pipeline(['Battlefield', 'battlefield', 'battle field', 'батлфилд', 'батлфилт', 'батла']).interpretation(Game.name),
    morph_pipeline([ '2', '1', 'Bad company', '4', '3']).interpretation(Game.second_name), 
    morph_pipeline(['2']).interpretation(Game.number).optional()) 

GTA  = rule(
    morph_pipeline(['GTA', 'gta', 'Gta', 'гта', 'гта4', 'гта5', 'ГТА']).interpretation(Game.name),
    morph_pipeline([ '4', 'V', '5', 'сан андрес']).interpretation(Game.second_name), 
    morph_pipeline(['the lost and damned']).interpretation(Game.number).optional())
    



GAME = or_(COD, Borderlands, AC, ME, Witcher, FIFA).interpretation(Game)

parser = Parser(GAME) 

In [0]:
matches = []

for sent in data.text[:9000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [54]:
for m in matches:
    print(m.name, m.second_name)

фифа 15
асассин 4
видмак 3
FIFA 16
Call of Duty Ghosts
Call of Duty 4
Assassins Creed 4
Fifa 11
Fifa 15
Mass Effect 3
Assassins creed unity
FIFA 16
assassins creed синдикат
FIFA 13
FIFA 16
FIFA 16
FIFA 15
Mass Effect 3
Fifa 15
FIFA 16
FIFA 15
Ведьмак 3
Mass effect 2
fifa 16
borderlands 2
ведьмак 2
Mass Effect 3
ведьмак 3
fifa 14
FIFA 14
FIFA 15
FIFA 12
Borderland 2
Fifa 14
Fifa 16
fifa 15
FIFA 15
FIFA 15
fifa 15
Call of duty Black ops
Фифа 15
Call of Duty black ops
fifa 15
Ведьмак 3
Ведьмак 3
FIFA 15
ведьмак 3
FIFA 16
Ведьмак 3
Fifa 13
FIFA 16
FIFA 14
Ведьмак 3
фифа 15
fifa 16
Ведьмак 2
FIFA 15
Witcher 3
Fifa 15
Mass Effect 2
Mass Effect 3
Ведьмак 2
FiFa 13
Call of Duty Black Ops
Call of Duty Black Ops
Mass Effect 2
Borderlands 2
FIFA 15
Fifa 15
FIFA 16
Assassin Creed Синдикат
Fifa 16
CALL OF DUTY BLACK OPS
Call of Duty Black Ops
Assassins Creed 4
Assassins Creed 3
Fifa 14
ведьмак 3
Fifa 15
Ведьмак 3
fifa 16
Call of duty black ops
фифа 16
CALL of DUTY GHOSTS
Fifa 15
Fifa 14
Ведьмак 2
f

In [0]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [0]:
for m in matches:
    print(m.name, m.second_name)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox One
Playstation 4
Xbox 360
Xbox One
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox One
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox One
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox One
Xbox ONE
Playstation 2
Playstation 4
Xbox One
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


В общем случае к нормальной форме приводит .normalized() добавленный к атрибуту в интерпретации. Но он работает через pymorphy2, а он работает с русским языком. Поэтому в нашем случае его можно использовать только для привода к нижнему регистру.

Когда у каждого и слов в газзетире есть своя нормальная форма, можно сделать газзетир словарём, где ключи это нужные формы, а значения - нормальные формы. А в интерпретации к .normalized() добавить .custom() и через него дергать нужную правильную форму.

In [0]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model.normalized()),
    in_caseless('sxe').interpretation(Pristavka.version.normalized()).optional()) 

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    # для простоты можно написать вот так
    in_('1234').interpretation(Pristavka.model.normalized()),
    #изменения вот тут                                                      вот тут дергаем правильную форму
    caseless_pipeline(VERSIONS).interpretation(Pristavka.version.normalized().custom(VERSIONS.get)).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [0]:
list(parser.findall('PS 3 SlIm'))[0].fact

Pristavka(name='Playstation',
          model='3',
          version='Slim')

In [0]:
list(parser.findall('XbOx 360 X'))[0].fact

Pristavka(name='Xbox',
          model='360',
          version='x')

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [0]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox one
Playstation 4
Xbox 360
Xbox one
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Playstation 4
Xbox 360
Xbox one
Playstation 4
Playstation 4
Xbox 360
Playstation 3
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox one
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Xbox 360
Xbox one
Xbox one
Playstation 2
Playstation 4
Xbox one
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4


Улучшения для этого типа сущностей довольно очевидные и простые - нужно расширять газзетиры, можно добавить другие приставки вроде денди и нинтендо.

Для сущностей, которые нельзя перечислить словарём, нужны контекстные правила. В целом они не сильно сложнее того, что мы сейчас написали. Возможно вам понадобятся другие полезные штуки из Yargy (предикат - and_, учёт грамматической информации - gram('NOUN'), согласование по роду, числу и падежу - gnc).

Учить сразу все не имеет смысла. Возьмите практическую задачу, документацию к Yargy (https://yargy.readthedocs.io/ru/latest/index.html) и разбирайте только то, что вам нужно. 